In [ ]:
import time
import numpy as np
import pandas as pd
import math
import random
import torch

from finrl.apps import config
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.noise import (
    NormalActionNoise,
    OrnsteinUhlenbeckActionNoise,
)
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.finrl_meta.data_processors.processor_yahoofinance import YahooFinanceProcessor

from stable_baselines3.common.vec_env import DummyVecEnv
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

import datetime
import statistics
import seaborn as sns
import pyfolio
from pyfolio import timeseries

import sys
sys.path.append("../FinRL-Library")

from gym.utils import seeding
import gym
from gym import spaces


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def set_seed(seed_value):
    random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed(100)

In [ ]:
class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                reward_by = 'reward',
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.max_portfolio_value = 0        
        self.mdd = 0.0001
        self.reward_by = reward_by
        self.return_list = []
        self.pre_value = initial_amount


        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
#         print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            if self.reward_by=='reward':
                self.reward = new_portfolio_value 
            # self.reward = (new_portfolio_value-self.initial_amount)/self.initial_amount
            elif self.reward_by=='mdd':
                now_return = (new_portfolio_value-self.initial_amount)/self.initial_amount
                ann_return = (1+now_return) ** (252/self.day) -1
                if self.max_portfolio_value <  new_portfolio_value :
                    self.max_portfolio_value = new_portfolio_value
                now_dd = 1-(new_portfolio_value/self.max_portfolio_value)
                if self.mdd < now_dd:
                    self.mdd = now_dd   
                self.reward = 1/(self.mdd+0.0001)
            elif self.reward_by=='1mstdev':
                rrr = (new_portfolio_value-self.pre_value)/self.pre_value
                self.return_list.append(rrr)
#                 print(self.return_list)
                if len(self.return_list)<21:
                    self.reward = 0
                else:
                    self.return_list.pop(0)
                    stdev = statistics.stdev(self.return_list)
                    if stdev==0:
                        self.reward = 1/(stdev+0.0001)
                    else:
                        self.reward = 1/stdev
#             self.reward = ann_return+(1/self.mdd)
#             print(ann_return)
#             print("Step reward: ", self.reward)
#             self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        seed = 100
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [ ]:
def get_data(etfs):
    dp = YahooFinanceProcessor()
    df = dp.download_data(start_date = '2001-01-01',
                     end_date = '2021-12-31',
                     ticker_list = etfs, time_interval='1D')
    fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

    df = fe.preprocess_data(df)
    # add covariance matrix as states
    df=df.sort_values(['date','tic'],ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)

        covs = return_lookback.cov().values 
        cov_list.append(covs)

    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    print('targets:',df.tic.unique())
    return df

In [ ]:
def train_rl(df,start_date,end_date,initial_amount=1000000):

    
    train = data_split(df, start_date,end_date)
    stock_dimension = len(train.tic.unique())
    state_space = stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
    
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4
    }
    
    e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    
    agent = DRLAgent(env = env_train)
    PPO_PARAMS = {
        "n_steps": 2048,
        "ent_coef": 0.005,
        "learning_rate": 0.0001,
        "batch_size": 128,
    }
    model_ppo = agent.get_model("ppo",seed=100,model_kwargs = PPO_PARAMS)
#     enable_dropout(model_ppo)
    trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)
#     trained_ppo.save('trained_models/trained_ppo.zip')
    return trained_ppo

In [ ]:
def trade_rl(df,start_date,end_date,trained_model,initial_amount=1000000):
    trade = data_split(df,start_date,end_date)
    stock_dimension = len(trade.tic.unique())
    state_space = stock_dimension
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4
    }
    e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
    df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_model, environment = e_trade_gym)
    return df_daily_return, df_actions

In [ ]:
def get_ABC(df_ttt):
    df_ttt = df_ttt.reset_index()
    df_ttt['day_return'] = 0 
    for i in range(len(df_ttt)-1):
        df_ttt.loc[i+1,'day_return'] = (df_ttt['Close'][i+1] - df_ttt['Close'][i])/df_ttt['Close'][i]
    
    df_ttt = df_ttt.fillna(0)
    df_ttt['max']=0
    s1 = df_ttt['Close']
    for i in range(len(df_ttt)):
        df_ttt.loc[i,'max'] = s1[0:i+1].max() 
    
    df_ttt['dd'] = 0
    df_ttt['dd'] = 1-(df_ttt['Close']/df_ttt['max'])
    
    mdd = df_ttt['dd'].max()

    df_ttt['total_value'] = 1000000
    for i in range(1,len(df_ttt)):
        df_ttt.loc[i,'total_value'] = df_ttt['total_value'][i-1]*(df_ttt['day_return'][i]+1)

    stdev_ttt = statistics.stdev(df_ttt['day_return'])* math.pow( 252, 0.5 )

    reward_ttt = (df_ttt['total_value'][len(df_ttt)-1]/df_ttt['total_value'][0])**(252/len(df_ttt))-1

    return reward_ttt,stdev_ttt,mdd

def get_avg_ABC(comb,start,end):
    w = [0.36,0.18,0.06,0.4]
    df_ttt_close = pd.read_csv('all_etf_close.csv')#這個檔案要重做
    col = comb.copy()
    col.insert(0,'Date')
    ttttt = df_ttt_close[col]
    ttttt = ttttt[(ttttt['Date']>=start) & (ttttt['Date']<end) ]
    ttttt['Close'] = 0
    c=0
    for name in comb:
        print(name)
        ttttt['Close']+=(ttttt[name]*w[c])
        c+=1
    ttttt['Close'] /= len(comb)
    ttttt = ttttt.reset_index()
#     print(ttttt)
    reward_ttt,stdev_ttt,mdd = get_ABC(ttttt)
    return reward_ttt,stdev_ttt,mdd
# get_avg_ABC(['ITOT','VEU','VNQ','AGG'],'2016-01-01','2021-01-01')

In [ ]:

def detect_uncertainty(models,org_df):
    train_action_list = []
    train_return_list = []
#     trained_model = models[0]
    for j in range(250):
        print((j))
        action_tmp = []
        reward_tmp = []
        for i in range(len(reward_by_list)):
            trained_model = models[i]
            train_df_daily_return, train_df_actions = trade_rl(org_df,org_train_start,org_train_end,trained_model,org_initial_amount)
            action_tmp.append(train_df_actions)
            train_df_daily_return['daily_return']+=1
            reward_tmp.append(train_df_daily_return)
    
        df_concat = pd.concat(action_tmp)
        by_row_index = df_concat.groupby(df_concat.index)
        final_action = by_row_index.mean()
        train_action_list.append(final_action)
        
        df_concat = pd.concat(reward_tmp)
        by_row_index = df_concat.groupby(df_concat.index)
        final_reward = by_row_index.mean()-1
        train_return_list.append(final_reward)
    
    df_concat = pd.concat(train_action_list)
    by_row_index = df_concat.groupby(df_concat.index)
    mean_df = by_row_index.mean()
    stdev_df = by_row_index.std()
    mean_dict = mean_df.T.to_dict('list')
    stdev_dict = stdev_df.T.to_dict('list')
    stdev_list = list(stdev_dict.values())
    mean_list = list(mean_dict.values())
    
    transp_stdev_list = np.array(stdev_list).T
    sig = statistics.stdev(transp_stdev_list.reshape(-1))
    mu = sum(transp_stdev_list.reshape(-1))/len(transp_stdev_list.reshape(-1))
    thresh = mu+sig*2.5
    print(thresh)
    
    
    trade_action_list = []
    trade_return_list = []
    trained_model = models[0]
    for j in range(250):
        print((j))
        action_tmp = []
        reward_tmp = []
        for i in range(len(reward_by_list)):
            trained_model = models[i]
            trade_df_daily_return, trade_df_actions = trade_rl(org_df,org_trade_start,org_trade_end,trained_model,org_initial_amount)
            action_tmp.append(trade_df_actions)
            trade_df_daily_return['daily_return']+=1
            reward_tmp.append(trade_df_daily_return)
    
        # trade_dates = list(trade_df_daily_return['date'])

        df_concat = pd.concat(action_tmp)
        by_row_index = df_concat.groupby(df_concat.index)
        final_action = by_row_index.mean()
        trade_action_list.append(final_action)
        
        df_concat = pd.concat(reward_tmp)
        by_row_index = df_concat.groupby(df_concat.index)
        final_reward = by_row_index.mean()-1
        trade_return_list.append(final_reward)
        
    df_concat = pd.concat(trade_action_list)
    by_row_index = df_concat.groupby(df_concat.index)
    mean_df = by_row_index.mean()
    stdev_df = by_row_index.std()
    
    detect = []
    for d in stdev_df.T:
        for etf in stdev_df.columns:
            stdev = stdev_df[etf][d]
            mean = mean_df[etf][d]
            if stdev>thresh:
                detect.append([d,etf,stdev,mean])
    print(detect)
    
    return detect,df_concat,trade_df_daily_return


In [ ]:

def org_detect_func(org_etfs):
    org_df = get_data(org_etfs)
    models = []
    
    for i in range(len(reward_by_list)):
        print(i)
        trained_model = train_rl(org_df,org_train_start,org_train_end,org_initial_amount,reward_by_list[i])
        models.append(trained_model)
        trained_model.save('./models/model_'+reward_by_list+'.pt')
    
    detect,df_concat,trade_df_daily_return = detect_uncertainty(models,org_df)
    
    ddate = detect[0][0]
    dtarget = detect[0][1]
    textfile = open("./detect_record/mean_stdev.txt", "a")
    textfile.write('abnormal'+'\t'+ddate+'\t'+dtarget+"\n")
    
    df_concat_new = df_concat.reset_index()
    df_concat_cut = df_concat_new[df_concat_new['date']==detect[0][0]]
    fig,ax=plt.subplots(1,len(org_etfs),figsize=(30,8))
    i=0
    for col in df_concat_cut.columns[1:]:
        tttt = list(df_concat_cut[col])
        stdev = statistics.stdev(tttt)
        mean = sum(tttt)/len(tttt)
        print(i,col,stdev,mean)
        textfile.write(col+'\t'+str(stdev)+'\t'+str(mean)+"\n")
        sns.distplot(tttt,ax = ax[i],bins=10 )
        i+=1
    plt.savefig('./fig/'+detect[0][0]+'_abnormal.png')
    plt.show()
    plt.close()
    textfile.close()

    return models,detect,trade_df_daily_return


In [ ]:


def find_new_target(old_etfs,detect,detect_latest,models,last_detect_date):

    change_date = detect_latest[0]
    change_etf = detect_latest[1]
    change_etf_idx = old_etfs.index(change_etf)
    change_date_org = str(change_date)
    
    change2org=False
    if old_etfs!=org_etfs:
        for i in range(len(org_etfs)):
            etf = org_etfs[i]
            now_etfs = old_etfs.copy()
            print(etf,now_etfs)
            if etf not in now_etfs:#org誰被換掉
                print(etf+'not in comb')
                for j in range(len(now_etfs)):
                    if now_etfs[j] not in org_etfs:
                        now_etfs[j] = etf
                        break
                print(now_etfs)
                dt_tmp = last_detect_date.split('-')
                dt = datetime.date(int(dt_tmp[0]),int(dt_tmp[1]),int(dt_tmp[2]))
                time_del = datetime.timedelta(days=1) 
                dt = dt+time_del
                dt_tmp = change_date.split('-')
                dt_stop = datetime.date(int(dt_tmp[0]),int(dt_tmp[1]),int(dt_tmp[2]))
                print(dt,dt_stop)
                while True:
                    if dt==dt_stop:
                        break
                    if str(dt) not in trade_dates:
                        print(dt,'no trade')
                        dt = dt+time_del
                        continue
                    avg_reward,avg_std,avg_mdd = get_avg_ABC(now_etfs,org_trade_start,str(dt))
#                     if avg_reward>0.045 and avg_std<0.0832 and avg_mdd<0.1103:
                    if avg_reward>0.063 and avg_std<0.113 and avg_mdd<0.145:#這裡要手調
                        print(dt,'abc ok')
                        change2org=True
                        break
                    dt = dt+time_del
                    print(dt)
            if change2org:
                break
                
    change_success = False
    for etf in all_etf:
        if not change2org:
            now_etfs = old_etfs.copy()
            change_date = change_date_org
            if etf not in now_etfs:
                now_etfs[change_etf_idx] = etf
                
            else:
                continue
            
        else:
            change2org=False
            change_date = str(dt)

        print(now_etfs)
        avg_reward,avg_std,avg_mdd = get_avg_ABC(now_etfs,'2016-01-01',change_date)
        if avg_reward>0.06 and avg_std<0.12 and avg_mdd<0.15:#0.0862 0.1133 #這裡要手調
#         if avg_reward>0.0415 and avg_std<0.1 and avg_mdd<0.13:#0.0862 0.1133
            print('abc ok')
        else:
            continue
    

        now_df = get_data(now_etfs)
        print(now_df.tic.unique())
        if len(now_df.tic.unique())!=len(org_etfs):
            continue

        detect_new,df_concat,_ = detect_uncertainty(models,now_df)
        
        #如果非異常 跳出迴圈 回傳這個組合與模型
        flag = True
        detect_latest = []
        for i in range(len(detect_new)):
            dt_tmp = change_date.split('-')
            dt = datetime.date(int(dt_tmp[0]),int(dt_tmp[1]),int(dt_tmp[2]))
            
            dt_tmp = detect_new[i][0].split('-')
            dt_new = datetime.date(int(dt_tmp[0]),int(dt_tmp[1]),int(dt_tmp[2]))
            
            if dt==dt_new and detect_new[i][1]==change_etf: #if detect_new[i][0]==change_date:
                flag=False
                print('detect_new[i]',detect_new[i])
                break
            elif dt==dt_new:
                detect_latest = detect_new[i]
                print('detect_latest',detect_latest)
                break
            elif dt_new>dt: #dt_new較晚
                detect_latest = detect_new[i]
                print('detect_latest',detect_latest)
                break
               
        if flag:
            change_success=True
            print(now_etfs,'successed','normal@',change_date)
            # plot ok distribution
            textfile = open("./detect_record/mean_stdev.txt", "a")
            textfile.write('normal'+'\t'+change_date+"\n")
            
            df_concat_new = df_concat.reset_index()
            df_concat_cut = df_concat_new[df_concat_new['date']==change_date]
            fig,ax=plt.subplots(1,len(now_etfs),figsize=(30,8))
            i=0
            for col in df_concat_cut.columns[1:]:
                print(df_concat_cut[col])
                tttt = list(df_concat_cut[col])
                stdev = statistics.stdev(tttt)
                mean = sum(tttt)/len(tttt)
                print(i,col,stdev,mean)
                textfile.write(col+'\t'+str(stdev)+'\t'+str(mean)+"\n")
                sns.distplot(tttt,ax = ax[i],bins=10 )
                i+=1
            plt.savefig('./fig/'+change_date+'_'+change_etf+'_ok.png')
            plt.show()
            plt.close()
#             textfile.close()
  
            try:
                # plot abnormal distribution
                textfile.write('abnormal'+'\t'+detect_latest[0]+'\t'+detect_latest[1]+"\n")
            
                df_concat_new = df_concat.reset_index()
                df_concat_cut = df_concat_new[df_concat_new['date']==detect_latest[0]]
                fig,ax=plt.subplots(1,len(now_etfs),figsize=(30,8))
                i=0
                for col in df_concat_cut.columns[1:]:
                    tttt = list(df_concat_cut[col])
                    stdev = statistics.stdev(tttt)
                    mean = sum(tttt)/len(tttt)
                    print(i,col,stdev,mean)
                    textfile.write(col+'\t'+str(stdev)+'\t'+str(mean)+"\n")
                    sns.distplot(tttt,ax = ax[i],bins=10 )
                    i+=1
                plt.savefig('./fig/'+detect_latest[0]+'_'+detect_latest[1]+'_abnormal.png')
                plt.show()
                plt.close()
                textfile.close()
            
                break
            except:
                pass
        else:
            print(now_etfs,'failed')
    if change_success:
        return models,now_etfs,detect_new,detect_latest,change_date
    else:
        return models,None,None,None,None


In [ ]:
# 重新生成2001就存在的
filename = 'all_us_etf.txt'#

all_etf = []  
f = open(filename)
for line in f:
#     print(line[:-1])
    all_etf.append(line[:-1])
print(len(all_etf))

### 初始化一些參數

In [ ]:
reward_by_list=['reward','mdd','1mstdev']
org_etfs = ['ITOT', 'VEU', 'VNQ', 'AGG']
# org_etfs = ['VTI','TLT','IEF','GLD','DBC']
org_train_start = '2009-01-01'
org_train_end = '2016-01-01'
org_trade_start = '2016-01-01' 
org_trade_end = '2021-01-01'
org_initial_amount = 1000000

etf_record = [org_etfs]
detect_record = []
detect_date_record = []

### 訓練模型+第一次偵測+儲存資訊

In [ ]:
models,org_detect,trade_df_daily_return_org = org_detect_func(org_etfs)
trade_dates = list(trade_df_daily_return_org['date'])
detect = org_detect.copy()
detect_record.append(detect)
detect_date_record.append(detect[0][0])

In [ ]:
textfile = open("./detect_record/detect_record.txt", "w")
textfile.write("00"+'\n')
for dtct_r in detect:
    for kkk in range(len(dtct_r)):
        element = dtct_r[kkk]
#         print(element)
        textfile.write(str(element))
        if kkk<len(dtct_r)-1:
            textfile.write("\t")
        else:
            textfile.write("\n")
textfile.close()

textfile = open("./detect_record/detect_date_record.txt", "w")
textfile.write(detect[0][0]+'\n')
textfile.close()

### 開始偵測+更換

In [ ]:
### 還沒解決如果都找不到 回傳None的問題
old_etfs = org_etfs.copy()
detect = detect_record[0]
detect_latest = detect[0]
last_detect_date = '2016-01-04'# test 第一天日期
# print(detect)
count=1
while True:
    random.shuffle(all_etf)
    print('old_etfs:',old_etfs)
    print('detect',detect)
    trained_model,now_etfs,detect_new,detect_latest,last_detect_date = find_new_target(old_etfs,detect,detect_latest,trained_model,last_detect_date)
    detect_record.append(detect_new)
    detect_date_record.append(detect_latest[0])
    etf_record.append(now_etfs)
    
#     last_detect_date = detect_latest[0]
    old_etfs = now_etfs.copy()
    detect = detect_new
    
    textfile = open("./detect_record/detect_record.txt", "a")
    if count<10:
        textfile.write('0'+str(count)+'\n')
    else:
        textfile.write(str(count)+'\n')
    for dtct_r in detect:
        for kkk in range(len(dtct_r)):
            element = dtct_r[kkk]
#             print(element)
            textfile.write(str(element))
            if kkk<len(dtct_r)-1:
                textfile.write("\t")
            else:
                textfile.write("\n")
    textfile.close()
    
    textfile = open("./detect_record/detect_date_record.txt", "a")
    textfile.write(detect_latest[0]+'\n')
    textfile.close()
    
    count+=1
    if detect_latest==[]:
        break

### 都換完了 讀取更換的時間和標的

In [ ]:
filename = './detect_record/mean_stdev.txt'#


f = open(filename)
flag = False
dt_rcrd = []
etf_rcrd = []
tmp=[]
for line in f:
#     print(line[:-1])
    ttt = line[:-1]
    ttt_list = ttt.split('\t')
    print(ttt_list)
    if flag:
        tmp.append(ttt_list[0])
        print(tmp)
        if len(tmp)==4:
            etf_rcrd.append(tmp)
            tmp=[]
    if ttt_list[0]=='normal':
#         print('normal')
        flag=True
        tmp=[]
        dt_rcrd.append(ttt_list[1])
    if ttt_list[0]=='abnormal':
        flag = False
print(dt_rcrd)
print(etf_rcrd)
print(len(dt_rcrd),len(etf_rcrd))

### 每個時間和組合都重新trade一次

In [ ]:
trade_action_list = []
trade_return_list = []
for i in range(len(etf_rcrd)):#etf_record
    now_etfs = etf_rcrd[i]
    now_df = get_data(now_etfs)
    print(now_df.tic.unique())
    trade_df_daily_return, trade_df_actions = trade_rl(now_df,org_trade_start,org_trade_end,trained_model,org_initial_amount)
    trade_action_list.append(trade_df_actions)
    trade_return_list.append(trade_df_daily_return)


### combine成一個return df

In [ ]:
concate = []
detect_date_record_new = dt_rcrd.copy()#detect_date_record
detect_date_record_new.insert(0, "2016-01-04")
for i in range(len(detect_date_record_new)):
    if i==0:
        trade_df_daily_return_org = trade_return_list[i]
    if i!=len(detect_date_record_new)-1:
        trade_start = detect_date_record_new[i] 
        trade_end = detect_date_record_new[i+1]
        trade_df_daily_return = trade_return_list[i]
        start_idx = trade_df_daily_return[trade_df_daily_return['date'] == trade_start].index.tolist()[0]
        end_idx = trade_df_daily_return[trade_df_daily_return['date'] == trade_end].index.tolist()[0]
        df_toconcate = trade_df_daily_return[start_idx:end_idx]
    else:
        trade_start = detect_date_record_new[i] 
        trade_df_daily_return = trade_return_list[i]
        start_idx = trade_df_daily_return[trade_df_daily_return['date'] == trade_start].index.tolist()[0]
        df_toconcate = trade_df_daily_return[start_idx:]
    concate.append(df_toconcate)
new_trade_df_daily_return = pd.concat(concate,axis=0,ignore_index = True)
new_trade_df_daily_return

### 輸出績效

In [ ]:
DRL_strat = convert_daily_return_to_pyfolio_ts(new_trade_df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
print("==============New DRL Strategy Stats===========")
perf_stats_all

In [ ]:
DRL_strat = convert_daily_return_to_pyfolio_ts(pd.read_csv(('./csv/classic_org.csv'))[['date','daily_return']])#org_trade_df_daily_return
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
print("==============Org DRL Strategy Stats===========")
perf_stats_all

In [ ]:
now_df = get_data(org_etfs)
print(now_df.tic.unique())
train_df_daily_return, train_df_actions = trade_rl(now_df,org_train_start,org_train_end,trained_model,org_initial_amount)
    
DRL_strat = convert_daily_return_to_pyfolio_ts(train_df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")
print("==============Original DRL Strategy Stats @ train===========")
perf_stats_all

In [ ]:
DRL_strat_org = convert_daily_return_to_pyfolio_ts(trade_df_daily_return_org)

DRL_strat = convert_daily_return_to_pyfolio_ts(new_trade_df_daily_return)

print("==============DRL Strategy Stats===========")
#     print(perf_stats_all)
    
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=DRL_strat_org, set_context=False)


### 存df

In [ ]:
new_trade_df_daily_return.to_csv('./csv/classic_bound_change_new.csv')
trade_return_list[0].to_csv('./csv/classic_bound_org.csv')
for i in range(len(trade_return_list)):
    if i<10:
        iiiii = '0'+str(i)
    else:
        iiiii = str(i)
    trade_return_list[0].to_csv('./csv/classic_bound_new_'+iiiii+'.csv')